# CGM Analysis

### Tasks
1. Extract 4 (one for each student) different types of time series features from only the CGM data cell array and CGM timestamp cell array (10 points each) total 40
2.	For each time series explain why you chose such feature (5 points each) total 20
3.	Show values of each of the features and argue that your intuition in step b is validated or disproved? (5 points each ) total 20
4.	Create a feature matrix where each row is a collection of features from each time series. SO if there are 75 time series and your feature length after concatenation of the 4 types of featues is 17 then the feature matrix size will be 75 X 17 (10 points)
5.	Provide this feature matrix to PCA and derive the new feature matrix. Chose the top 5 features and plot them for each time series.  (5 points)
6.	For each feature in the top 5 argue why it is chosen as a top five feature in PCA? (3 points each) total 15


In [1]:
from datetime import datetime, timedelta
import pandas as pd
import pandas
import matplotlib.pyplot as plt

## Read Input file

Make sure the path is correct for you

In [2]:
cgm_timestamps = pd.read_csv("dm_proj1_data/CGMDatenumLunchPat1.csv")
cgm_values = pd.read_csv("dm_proj1_data/CGMSeriesLunchPat1.csv")

### Reference function

In [3]:
def mdate_to_pdate(mdate):
    return (datetime.fromordinal(int(mdate)) + timedelta(days=mdate % 1) - timedelta(days=366)).strftime("%Y-%m-%d %H:%M:%S")

In [4]:
# first and last
print('first',mdate_to_pdate(737225.5842))
print('last',mdate_to_pdate(737225.48))

first 2018-06-14 14:01:14
last 2018-06-14 11:31:11


### Pre-processing

1. Convert Datetimes to useable format
2. Merge Timestamps and Values into one dataframe
3. Rearrange columns w.r.t column names like timeseries#_x and timeseries#_y

In [5]:
#cgm_timestamps.interpolate(method = 'quadratic')

In [6]:
#cgm_timestamps

In [7]:
for col in cgm_timestamps.columns:
    cgm_timestamps[col] = cgm_timestamps[col].apply(lambda x: mdate_to_pdate(x) if pd.notnull(x) else x)

In [8]:
cgm_timestamps, cgm_values = cgm_timestamps.T, cgm_values.T

In [9]:
cgm_timestamps = cgm_timestamps.set_index([pandas.Index(['cgm_{}'.format(i) for i in range(1, len(cgm_timestamps) + 1)])])
cgm_values = cgm_values.set_index([pandas.Index(['cgm_{}'.format(i) for i in range(1, len(cgm_timestamps) + 1)])])

In [10]:
cgm_merged = pandas.merge(cgm_timestamps, cgm_values, how="inner", left_index=True, right_index=True)

In [11]:
cols = cgm_merged.columns.tolist()
new_cols = []
for i in range(len(cgm_merged)):
    new_cols.append(cols[0 + i])
    new_cols.append(cols[len(cgm_merged.columns) // 2 + i])

In [12]:
cgm_merged = cgm_merged[new_cols]
cgm_merged = cgm_merged.fillna(method='ffill', limit=2)

In [13]:
# sort timeseries
for i in cgm_merged.columns:
    cgm_merged[i] = cgm_merged[i].values[::-1]

In [14]:
cgm_merged.to_csv("dm_proj1_data/intermediate_files/cgm_merged_2.csv")

### Plot all Meals w.r.t time in descending order

In [15]:
for i in range(len(cgm_merged)):
    x, y = str(i) + "_x", str(i) + "_y"
    plt.plot(cgm_merged[x], cgm_merged[y])
    plt.title("Patient: 1 Meal #{}".format(x.split("_")[0]))
    plt.xticks(rotation=90)
    plt.savefig("Meal_{}.png".format(x.split("_")[0]))
    plt.close()

In [27]:
num = 2
for i in range(len(cgm_merged.columns) // 2):
    cgm_merged.insert(num, '{}_out_of_range'.format(i), [70 - j if j < 70 else j - 180 if j > 180 else 0 for j in cgm_merged['{}_y'.format(i)]], True)
    num += 3
cgm_merged

,0_x,0_y,0_out_of_range,1_x,1_y,1_out_of_range,2_x,2_y,2_out_of_range,3_x,...,27_out_of_range,28_x,28_y,28_out_of_range,29_x,29_y,29_out_of_range,30_x,30_y,30_out_of_range
cgm_1,2018-06-14 11:31:11,91.0,0.0,2018-06-06 12:38:59,216.0,36.0,2018-06-05 10:48:54,123.0,0.0,2018-06-04 11:13:49,...,5.0,2017-11-14 11:33:28,111.0,0.0,2017-11-11 11:57:20,198.0,18.0,2017-11-09 11:27:08,100.0,0
cgm_2,2018-06-14 11:36:10,92.0,0.0,2018-06-06 12:38:59,216.0,36.0,2018-06-05 10:48:54,123.0,0.0,2018-06-04 11:18:49,...,5.0,2017-11-14 11:33:28,111.0,0.0,2017-11-11 11:57:20,198.0,18.0,2017-11-09 11:27:08,100.0,0
cgm_3,2018-06-14 11:41:10,91.0,0.0,2018-06-06 12:44:00,232.0,52.0,2018-06-05 10:53:53,123.0,0.0,2018-06-04 11:23:48,...,0.0,2017-11-14 11:38:28,109.0,0.0,2017-11-11 12:02:20,193.0,13.0,2017-11-09 11:32:08,95.0,0
cgm_4,2018-06-14 11:46:11,90.0,0.0,2018-06-06 12:49:00,245.0,65.0,2018-06-05 10:58:53,123.0,0.0,2018-06-04 11:28:48,...,0.0,2017-11-14 11:43:29,109.0,0.0,2017-11-11 12:07:21,182.0,2.0,2017-11-09 11:37:08,91.0,0
cgm_5,2018-06-14 11:51:11,91.0,0.0,2018-06-06 12:53:59,261.0,81.0,2018-06-05 11:03:54,124.0,0.0,2018-06-04 11:33:49,...,0.0,2017-11-14 11:48:29,108.0,0.0,2017-11-11 12:12:21,186.0,6.0,2017-11-09 11:42:09,89.0,0
cgm_6,2018-06-14 11:56:10,91.0,0.0,2018-06-06 12:58:59,269.0,89.0,2018-06-05 11:08:54,126.0,0.0,2018-06-04 11:38:49,...,0.0,2017-11-14 11:53:28,107.0,0.0,2017-11-11 12:17:20,187.0,7.0,2017-11-09 11:47:09,92.0,0
cgm_7,2018-06-14 12:01:10,97.0,0.0,2018-06-06 13:03:59,275.0,95.0,2018-06-05 11:13:53,127.0,0.0,2018-06-04 11:43:48,...,0.0,2017-11-14 11:58:28,104.0,0.0,2017-11-11 12:22:20,182.0,2.0,2017-11-09 11:52:08,96.0,0
cgm_8,2018-06-14 12:06:10,99.0,0.0,2018-06-06 13:09:00,284.0,104.0,2018-06-05 11:18:53,132.0,0.0,2018-06-04 11:48:48,...,0.0,2017-11-14 12:03:29,113.0,0.0,2017-11-11 12:27:21,161.0,0.0,2017-11-09 11:57:08,103.0,0
cgm_9,2018-06-14 12:11:11,98.0,0.0,2018-06-06 13:14:00,294.0,114.0,2018-06-05 11:23:54,134.0,0.0,2018-06-04 11:53:49,...,0.0,2017-11-14 12:08:29,117.0,0.0,2017-11-11 12:32:21,154.0,0.0,2017-11-09 12:02:09,115.0,0
cgm_10,2018-06-14 12:16:11,99.0,0.0,2018-06-06 13:18:59,310.0,130.0,2018-06-05 11:28:54,136.0,0.0,2018-06-04 11:58:49,...,0.0,2017-11-14 12:13:28,114.0,0.0,2017-11-11 12:37:20,153.0,0.0,2017-11-09 12:07:09,126.0,0


In [196]:
cgm_merged.skew(axis = 0, bia)

0_y                0.117677
0_out_of_range     0.659232
1_y               -0.649898
1_out_of_range    -0.649898
2_y                0.048788
2_out_of_range     0.344805
3_y                0.138445
3_out_of_range     0.780534
4_y               -0.139671
4_out_of_range     0.000000
5_y               -0.160430
5_out_of_range     0.358637
6_y               -0.003908
6_out_of_range     2.584048
7_y                0.232065
7_out_of_range     0.840412
8_y               -0.097560
8_out_of_range     2.256916
9_y                0.535281
9_out_of_range     2.078327
10_y              -0.274297
10_out_of_range    0.102441
11_y               0.310936
11_out_of_range    0.832903
12_y              -0.060018
12_out_of_range    0.708415
13_y               0.040811
13_out_of_range    1.168833
14_y              -0.255234
14_out_of_range    0.000000
                     ...   
16_y              -0.014947
16_out_of_range    0.326623
17_y              -0.155197
17_out_of_range    0.893317
18_y               0

In [28]:
posCount, negCount = 0, 0
outOfRange = []
for i in range(len(cgm_merged.columns) // 3):
    for j in cgm_merged['{}_out_of_range'.format(i)]:
        if j > 0:
            posCount += 1
        if j < 0:
            negCount += 1
#     outOfRange.append([posCount,negCount])
    outOfRange.append(posCount)
    posCount, negCount = 0, 0

In [29]:
timeRange = ['{1} - {0}'.format(cgm_merged['{}_x'.format(i)][0], cgm_merged['{}_x'.format(i)][-1]) for i in range(len(cgm_merged.columns)//3)]

In [34]:
feature_matrix = pd.DataFrame({'timeRange':timeRange, 'outOfRange': outOfRange})

## MK test

The purpose of the Mann-Kendall (MK) test (Mann 1945, Kendall 1975, Gilbert 1987) is to statistically assess if there is a monotonic upward or downward trend of the variable of interest over time. A monotonic upward (downward) trend means that the variable consistently increases (decreases) through time, but the trend may or may not be linear. The MK test can be used in place of a parametric linear regression analysis, which can be used to test if the slope of the estimated linear regression line is different from zero. The regression analysis requires that the residuals from the fitted regression line be normally distributed; an assumption not required by the MK test, that is, the MK test is a non-parametric (distribution-free) test. Hirsch, Slack and Smith (1982, page 107) indicate that the MK test is best viewed as an exploratory analysis and is most appropriately used to identify stations where changes are significant or of large magnitude and to quantify these findings.

In [31]:
import numpy as np  
from scipy.stats import norm, mstats
def mk_test(x, alpha = 0.3):  
    """   
    Input:
        x:   a vector of data
        alpha: significance level (0.05 default)

    Output:
        trend: tells the trend (increasing, decreasing or no trend)
        h: True (if trend is present) or False (if trend is absence)
        p: p value of the significance test
        z: normalized test statistics 

    Examples
    --------
      >>> x = np.random.rand(100)
      >>> trend,h,p,z = mk_test(x,0.05) 
    """
    n = len(x)

    # calculate S
    s = 0
    for k in range(n-1):
        for j in range(k+1, n):
            s += np.sign(x[j] - x[k])

    # calculate the unique data
    unique_x = np.unique(x)
    g = len(unique_x)

    # calculate the var(s)
    if n == g:  # there is no tie
        var_s = (n*(n-1)*(2*n+5))/18
    else:  # there are some ties in data
        tp = np.zeros(unique_x.shape)
        for i in range(len(unique_x)):
            tp[i] = np.sum(x == unique_x[i])
        var_s = (n*(n-1)*(2*n+5) - np.sum(tp*(tp-1)*(2*tp+5)))/18

    if s > 0:
        z = (s - 1)/np.sqrt(var_s)
    elif s < 0:
        z = (s + 1)/np.sqrt(var_s)
    else: # s == 0:
        z = 0

    # calculate the p_value
    p = 2*(1-norm.cdf(abs(z)))  # two tail test
    h = abs(z) > norm.ppf(1-alpha/2)

    if (z < 0) and h:
        trend = 'decreasing'
    elif (z > 0) and h:
        trend = 'increasing'
    else:
        trend = 'no trend'

    return trend, h, p, z

In [49]:
#Range length depends on cgm_merged length, if adding a new column please change here
for i in range(len(cgm_merged.columns)//3):
    col = str(i) + '_y'
    print("Meal #{} :{}".format(i,mk_test(cgm_merged[col].values.ravel())))

Meal #0 :('increasing', True, 1.8851586958135158e-13, 7.356633605055599)
Meal #1 :('increasing', True, 0.1735474219255808, 1.3608932659599007)
Meal #2 :('increasing', True, 5.78244119253668e-10, 6.196249601798579)
Meal #3 :('increasing', True, 3.480504773278881e-11, 6.624659544479574)
Meal #4 :('increasing', True, 0.042385006152762195, 2.0297200256358088)
Meal #5 :('increasing', True, 1.2441243590899376e-09, 6.074451228725762)
Meal #6 :('increasing', True, 1.572592209120316e-06, 4.8017827944648745)
Meal #7 :('increasing', True, 2.96138669142465e-11, 6.648478531431981)
Meal #8 :('increasing', True, 3.597122599785507e-14, 7.574719551224234)
Meal #9 :('increasing', True, 0.10586210111578964, 1.617074960373713)
Meal #10 :('increasing', True, 8.364474782807108e-06, 4.455636261559555)
Meal #11 :('increasing', True, 4.782294560357059e-10, 6.226086691766546)
Meal #12 :('increasing', True, 2.8931790074793184e-08, 5.54778102597717)
Meal #13 :('increasing', True, 3.4499650474195676e-06, 4.6420337

In [41]:
#Insert mk_testt_trend
feature_matrix.insert(2, "mk_test_trend", [mk_test(cgm_merged[str(i) + '_y'].values.ravel())[0] for i in range(len(cgm_merged.columns)//3)])

In [43]:
#Insert mk_test_p_value: p value of the significance test
feature_matrix.insert(3, "mk_test_p_value", [mk_test(cgm_merged[str(i) + '_y'].values.ravel())[2] for i in range(len(cgm_merged.columns)//3)])

In [46]:
#Insert mk_test_z_value: normalized test statistics
feature_matrix.insert(4, "mk_test_z_value", [mk_test(cgm_merged[str(i) + '_y'].values.ravel())[3] for i in range(len(cgm_merged.columns)//3)])

In [47]:
feature_matrix.head()

,timeRange,outOfRange,mk_test_trend,mk_test_p_value,mk_test_z_value
0,2018-06-14 14:01:11 - 2018-06-14 11:31:11,14,increasing,1.885159e-13,7.356634
1,2018-06-06 15:04:00 - 2018-06-06 12:38:59,31,increasing,1.735474e-01,1.360893
2,2018-06-05 13:13:53 - 2018-06-05 10:48:54,16,increasing,5.782441e-10,6.196250
3,2018-06-04 13:43:48 - 2018-06-04 11:13:49,13,increasing,3.480505e-11,6.624660
4,2018-05-21 14:08:44 - 2018-05-21 11:43:45,0,increasing,4.238501e-02,2.029720
